In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [16]:
import fastai
from fastai import *
from fastai.vision import *

### Python methods
_dir()_ will give you the list of in scope variables
_globals()_ will give you a dictionary of global variables
_locals()_ will give you a dictionary of local variables

In [17]:
for i, el in enumerate(dir()):
    if i % 4 == 0: print()
    print('{:28}'.format(el), end="\t")


ActivationStats             	AdamW                       	AdaptiveConcatPool2d        	AffineFunc                  	
AffineMatrix                	AnnealFunc                  	Any                         	AnyStr                      	
ArgStar                     	AverageMetric               	BatchSampler                	BatchSamples                	
BnFreeze                    	BoolOrTensor                	ByteTensor                  	Callable                    	
Callback                    	CallbackHandler             	CallbackList                	Classes                     	
ClassificationInterpretation	Collection                  	ConvLearner                 	CoordFunc                   	
Counter                     	CrossEntropyFlat            	DATA_BASEDIR                	DATA_DIR                    	
DOWNLOAD_FOLDER             	DataBunch                   	DataFrame                   	DataFrameOrChunks           	
DataLoader                  	Dataset                     	Datas

In [18]:
import pathlib
FASTAI_FOLDER = str(pathlib.PurePath(fastai.__file__).parent)
!echo fastai folder:
!echo {FASTAI_FOLDER}
!echo
!echo fastai contents:
!ls {FASTAI_FOLDER}
!echo
!echo --------------
!echo fastai __init__.py exports:
!echo --------------
!cat {FASTAI_FOLDER}/__init__.py
!echo 
!echo --------------
!echo fastai.vision exports
!echo --------------
!cat {FASTAI_FOLDER}/vision/__init__.py

fastai folder:
/home/oroneki/anaconda3/envs/dl/lib/python3.7/site-packages/fastai

fastai contents:
basic_data.py	collab.py    imports	  __pycache__	 train.py
basic_train.py	core.py      __init__.py  tabular	 utils
callback.py	datasets.py  layers.py	  text		 version.py
callbacks	gen_doc      metrics.py   torch_core.py  vision

--------------
fastai __init__.py exports:
--------------
from .basic_train import *
from .callback import *
from .callbacks import *
from .core import *
from .basic_data import *
from .layers import *
from .metrics import *
from .torch_core import *
from .train import *
from .gen_doc.nbdoc import doc
from .datasets import *
from .utils.collect_env import *
from .version import __version__

--------------
fastai.vision exports
--------------
from .learner import *
from .data import *
from .image import *
from .transform import *
from .tta import *
from . import models


### dataset: 
#### http://www.robots.ox.ac.uk/~vgg/data/pets/

In [19]:
help(untar_data)

Help on function untar_data in module fastai.datasets:

untar_data(url: str, fname: Union[pathlib.Path, str] = None, dest: Union[pathlib.Path, str] = None)
    Download `url` if doesn't exist to `fname` and un-tgz to folder `dest`



In [25]:
HOMEPATH_LIST = !echo $HOME
DATA_BASEDIR = pathlib.Path(HOMEPATH_LIST[0]) / 'data'
DATA_DIR = DATA_BASEDIR / 'oxford-iiit-pet'
print(DATA_BASEDIR, DATA_DIR)


/home/oroneki/data /home/oroneki/data/oxford-iiit-pet


In [26]:
DOWNLOAD_FOLDER = pathlib.Path(HOMEPATH_LIST[0]) / 'Downloads'

if not DOWNLOAD_FOLDER.exists():
    DOWNLOAD_FOLDER.mkdir()
    
if not DATA_BASEDIR.exists():
    DATA_BASEDIR.mkdir()


In [27]:
!ls {DOWNLOAD_FOLDER} | grep tgz

fastai_pets.tgz
ls: cannot access '/home/oroneki/data/oxford-iiit-pet': No such file or directory


In [28]:
dst = untar_data(URLs.PETS, fname=(DOWNLOAD_FOLDER / 'fastai_pets.tgz'), dest=DATA_DIR, ); dst
# untar_data ta miada
# - nao da erro se o fname for um diretorio existente
# - se especidicar uma pasta q enao existe em dest ele nao usa o mesmo nome da pasta e sim o nome original no folder acima e nao retorna o dest que ele mesmo escolheu
# - se a pasta existir ele num faz nada


PosixPath('/home/oroneki/data/oxford-iiit-pet')

## Jupyter Tip
If the Hbox widget doesnt work, close everything and run into the notebook env:
```jupyter labextension install @jupyter-widgets/jupyterlab-manager```

In [24]:
??untar_data

Signature: untar_data(url: str, fname: Union[pathlib.Path, str] = None, dest: Union[pathlib.Path, str] = None)
Source:   
def untar_data(url:str, fname:PathOrStr=None, dest:PathOrStr=None):
    "Download `url` if doesn't exist to `fname` and un-tgz to folder `dest`"
    dest = Path(ifnone(dest, _url2path(url)))
    fname = download_data(url, fname=fname)
    if not dest.exists(): tarfile.open(fname, 'r:gz').extractall(dest.parent)
    return dest
File:      ~/anaconda3/envs/dl/lib/python3.7/site-packages/fastai/datasets.py
Type:      function


In [30]:
DATA_DIR.ls()

['images', 'annotations']

In [31]:
path_anno = DATA_DIR / 'annotations'
path_img  = DATA_DIR / 'images'

In [32]:
fnames = get_image_files(path_img)
fnames[:5]

[PosixPath('/home/oroneki/data/oxford-iiit-pet/images/Maine_Coon_29.jpg'),
 PosixPath('/home/oroneki/data/oxford-iiit-pet/images/wheaten_terrier_38.jpg'),
 PosixPath('/home/oroneki/data/oxford-iiit-pet/images/basset_hound_149.jpg'),
 PosixPath('/home/oroneki/data/oxford-iiit-pet/images/Maine_Coon_32.jpg'),
 PosixPath('/home/oroneki/data/oxford-iiit-pet/images/scottish_terrier_133.jpg')]

In [34]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.jpg$'

In [37]:
?ImageDataBunch.from_name_re

Signature: ImageDataBunch.from_name_re(path: Union[pathlib.Path, str], fnames: Collection[pathlib.Path], pat: str, valid_pct: int = 0.2, test: str = None, **kwargs)
Docstring: <no docstring>
File:      ~/anaconda3/envs/dl/lib/python3.7/site-packages/fastai/vision/data.py
Type:      method


In [38]:
??ImageDataBunch.from_name_re

Signature: ImageDataBunch.from_name_re(path: Union[pathlib.Path, str], fnames: Collection[pathlib.Path], pat: str, valid_pct: int = 0.2, test: str = None, **kwargs)
Docstring: <no docstring>
Source:   
    @classmethod
    def from_name_re(cls, path:PathOrStr, fnames:FilePathList, pat:str, valid_pct:int=0.2, test:str=None, **kwargs):
        pat = re.compile(pat)
        def _get_label(fn): return pat.search(str(fn)).group(1)
        return cls.from_name_func(path, fnames, _get_label, valid_pct=valid_pct, test=test, **kwargs)
File:      ~/anaconda3/envs/dl/lib/python3.7/site-packages/fastai/vision/data.py
Type:      method


In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224)
data.normalize(imagenet_stats)